### 9.1.설치
- 텐서플로우는 수치연산을 기호로 표현한 그래프 구조를 만들고 처리한다는 기본 아이디어를 바탕으로 구현이 되었다. <br>
- 또한 알고리즘이 어떻게 돌아가고 있는지를 알려구지 위해 많은 정보를 모니터링하고 디스플레이 해주는 텐서보드 모듈이다. 
- 수식을 그래프로 표현하고 그래프를 세션에서 실행하여 결과를 얻는 작업을 거친다. 따라서 데이터를 node와 edge로 변환하여 플로우 그래프로 표현해야 한다. tensorflow에서는 node를 operation, edge를 tensor라고 한다. 
- 그래프에 오퍼레이션을 저장하고 각 데이터를 텐서로 연결하는 형태로 이루어진다. 
- tensorflow의 기본 idea는 노드를 만들고 edge로 연결하는 연산을 수행하는 것이다. 


In [2]:
%tensorflow_version 1.x #버전지정

`%tensorflow_version` only switches the major version: `1.x` or `2.x`.
You set: `1.x #버전지정`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [3]:
import tensorflow as tf
print(tf.__version__)

1.15.0


In [0]:
#간단한 예제
#식 3*2+1을 계산하는 경우
import tensorflow as tf

#오퍼레이션 생성
tensor_a=tf.constant(3, name='a')
tensor_b=tf.constant(2, name='b')
tensor_c=tf.constant(1,name='c')

tensor_d=tensor_a*tensor_b
tensor_e=tensor_d+tensor_c

In [5]:
tf.get_default_graph().get_operations()

[<tf.Operation 'a' type=Const>,
 <tf.Operation 'b' type=Const>,
 <tf.Operation 'c' type=Const>,
 <tf.Operation 'mul' type=Mul>,
 <tf.Operation 'add' type=AddV2>]

In [6]:
#세션으로 실행하여 결과확인
sess=tf.Session()
print(sess.run(tensor_e))

7


In [0]:
#결과확인
sess.close()

### 9.2.첫번째 계산그래프를 만들어 세션에서 실행하기

#### 텐서플로우 프로그램의 구성
- 구성단계: 훈련에 필요한 계산과 머신러닝 모델을 표현한 계산 그래프를 만든다<br>
- 실행단계: 훈련스탭을 반복해서 평가하고 반복루프를 진행

아래의 형태는 구성단계에 해당하는 코드이다. 실제 f식을 연산하는 것이 아니라, initialized값에 대해 훈련을 하는 stage에 올리기 전의 상태라고 보면 쉽다. 

In [0]:
import tensorflow as tf

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

In [9]:
f

<tf.Tensor 'add_2:0' shape=() dtype=int32>

이제 부터는 실행단계에 속하는 경우이다.실제 stage에서 연산의 단계에 올리며 실제 연산을 해주는 과정이다. 

### (1) . run session을 하나씩 설정하고 해제하는 경우 

In [10]:
sess = tf.Session()
sess.run(x.initializer) #변수 초기화 
sess.run(y.initializer)
result = sess.run(f) #평가 
print(result)

42


In [0]:
sess.close() #자원 해제

#### (2). with 선언문 사용하는 겨웅 

In [0]:
with tf.Session() as sess:
  x.initializer.run()
  y.initializer.run()
  result=f.eval()

#### (3).각 변수를 초기화하는 대신 global_variables_initializer()함수를 사용하는 경우

In [0]:
init=tf.global_variables_initializer() #변수의 초기화 
with tf.Session() as sess:
  init.run()
  result=f.eval()


In [14]:
result

42

#### (4). 자기자신을 한 세션으로 지정하는 interactiveSession을 사용한다.  

In [15]:
init=tf.global_variables_initializer()
sess=tf.InteractiveSession()
init.run()
result=f.eval()
print(result)

42


In [0]:
sess.close()

In [17]:
result

42

### 9.3.계산그래프 관리

In [18]:
#노드를 만들면 자동으로 기본 계산 그래프에 추가된다. 
#또한 노드가 중복되지 않도록 기본그래프 초기화 tf.reset_default_graph()해준다. 
x1= tf.Variable(1) #노드를 생성하는 경우
x1.graph is tf.get_default_graph() #해당 노드가 그래프에 있는지를 확인

True

독립적인 계산그래프를 여러개 만들어야 할 때 graph객체를 with블록 내에 임시로 만들어 임시 계산 그래프로 사용이 가능하다

In [0]:
 graph=tf.Graph()
 with graph.as_default():
   x2=tf.Variable(2)

In [20]:
x2.graph is graph

True

#### 9.4. 노드값의 생애주기
한 노드를 평가할 때 텐서플로우는 이 노드가 의존하고 있는 다른 노드들을 자동으로 찾아 먼저 평가한다. 

In [0]:
#오퍼레이션(각 노드에 대한 데이터)을 설정하는 방법
w = tf.constant(3) #구성
x = w + 2
y = x + 5
z = x * 3

In [22]:
with tf.Session() as sess: #실행
    print(y.eval())  #자동계산을 지원한다. 
    print(z.eval())  

10
15


### 9.5. 텐서플로우를 이용한 선형회귀
앞에서는 iteration을 돌리는 형태로 작성을 했었음. 여기서는 편향값을 넣고 신경망 형태로 작성을 한다. 

In [23]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing=fetch_california_housing() #데이터 
m,n=housing.data.shape
housing_data_plus_bias=np.c_[np.ones((m,1)), housing.data] #편향데이터를 넣은 형태로 데이터를 생성하는 경우 

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

In [0]:
with tf.Session() as sess:
    theta_value = theta.eval()

In [25]:
theta_value

array([[-3.67372932e+01],
       [ 4.37366009e-01],
       [ 9.47520509e-03],
       [-1.08159676e-01],
       [ 6.48537397e-01],
       [-3.84734449e-06],
       [-3.79239232e-03],
       [-4.19136107e-01],
       [-4.32144403e-01]], dtype=float32)

In [26]:
#numpy를 사용하여 구현했던 선형회귀의 경우 
X = housing_data_plus_bias
y = housing.target.reshape(-1, 1)
theta_numpy = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

print(theta_numpy)

[[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


In [27]:
#sklearn으로 구현했던 경우
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(housing.data, housing.target.reshape(-1, 1))

print(np.r_[lin_reg.intercept_.reshape(-1, 1), lin_reg.coef_.T])

[[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


### 9.6.경사하강법 구현


```python
#비용함수 계산이 가능한 함수 
def h(theta,X): #Linear hypothesis function
    return np.dot(X,theta)

def computeCost(mytheta,X,y): #Cost function
    """
    theta_start is an n- dimensional vector of initial theta guess
    X is matrix with n- columns and m- rows
    y is a matrix with m- rows and 1 column
    """
    #note to self: *.shape is (rows, columns)
    return float((1./(2*m)) * np.dot((h(mytheta,X)-y).T,(h(mytheta,X)-y)))

#Test that running computeCost with 0's as theta returns 32.07:
initial_theta = np.zeros((X.shape[1],1)) #(theta is a vector with n rows and 1 columns (if X has n features) )
print (computeCost(initial_theta,X,y))

#min cost의 theta값을 구하는 경우 
#Actual gradient descent minimizing routine
def descendGradient(X, theta_start = np.zeros(2)):
    """
    theta_start is an n- dimensional vector of initial theta guess
    X is matrix with n- columns and m- rows
    """
    theta = theta_start
    jvec = [] #Used to plot cost as function of iteration
    thetahistory = [] #Used to visualize the minimization path later on
    for meaninglessvariable in range(iterations):
        tmptheta = theta
        jvec.append(computeCost(theta,X,y))
        # Buggy line
        #thetahistory.append(list(tmptheta))
        # Fixed line
        thetahistory.append(list(theta[:,0]))
        #Simultaneously updating theta values
        for j in range(len(tmptheta)):
            tmptheta[j] = theta[j] - (alpha/m)*np.sum((h(theta,X) - y)*np.array(X[:,j]).reshape(m,1))
        theta = tmptheta
    return theta, thetahistory, jvec
```



In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

앞에서는 정규방정식을 이용하여 theta값을 구했다면 여기서는 gradient descent방법을 이용하여 optimized theta값을 구하게 된다. (tensorflow의 내장함수를 이용한다)

In [0]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

In [0]:
n_epochs=1000
learning_rate=0.01

#노드 생성
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y=tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

In [0]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

In [34]:

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

print("Best theta:")
print(best_theta)

Epoch 0 MSE = 2.754427
Epoch 100 MSE = 0.63222194
Epoch 200 MSE = 0.5727803
Epoch 300 MSE = 0.5585008
Epoch 400 MSE = 0.54907006
Epoch 500 MSE = 0.54228806
Epoch 600 MSE = 0.5373791
Epoch 700 MSE = 0.533822
Epoch 800 MSE = 0.53124255
Epoch 900 MSE = 0.5293704
Best theta:
[[ 2.0685525e+00]
 [ 7.7407807e-01]
 [ 1.3119237e-01]
 [-1.1784508e-01]
 [ 1.6477816e-01]
 [ 7.4407790e-04]
 [-3.9194509e-02]
 [-8.6135662e-01]
 [-8.2347977e-01]]


#### 9.6.2.자동미분 사용
- 단순 선형회귀에서는 MSE를 유도하는 것이 가능하지만 뉴럴 네트워크에서 이 비용함수를 유도하려면 계산식이 더욱 복잡해진다. 이에 따라 기호미분을 사용하여 자동으로 편미분 결과를 도출해낸다. 하지만 효율이 좋지 않음 
- 자동으로 그래디언트를 계산하는 방법은 후진모드 자동미분을 사용한다. (307참고하기)

In [0]:
#그래디언트 미분 가능한 4개의 방법 중에 후진모드 자동미분 방법을 사용한다. 
def my_func(a,b):
  z=0
  for i in range(100):
    z=a*np.cos(z+i)+z*np.sin(b-1)
  return z

In [36]:
my_func(0.2, 0.3)

0.04779633778994678

In [38]:
#tensorflow의 자동미분기능은 효율적으로 그래디언트를 계산한다. 
gradients=tf.gradients(mse, [theta])[0]
gradients

<tf.Tensor 'gradients_3/predictions_1_grad/MatMul_1:0' shape=(9, 1) dtype=float32>

In [0]:
#옵티마이저의 사용
#최적의 옵티마이저 
optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op=optimizer.minimize(mse)

In [40]:
#최적의 옵티마이저로 gradient descent를 실행
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

print("Best theta:")
print(best_theta)

Epoch 0 MSE = 2.754427
Epoch 100 MSE = 0.63222194
Epoch 200 MSE = 0.5727803
Epoch 300 MSE = 0.5585008
Epoch 400 MSE = 0.54907006
Epoch 500 MSE = 0.54228806
Epoch 600 MSE = 0.5373791
Epoch 700 MSE = 0.533822
Epoch 800 MSE = 0.53124255
Epoch 900 MSE = 0.5293704
Best theta:
[[ 2.0685525e+00]
 [ 7.7407807e-01]
 [ 1.3119237e-01]
 [-1.1784508e-01]
 [ 1.6477816e-01]
 [ 7.4407790e-04]
 [-3.9194509e-02]
 [-8.6135662e-01]
 [-8.2347977e-01]]


### 9.7.훈련알고리즘에 데이터 주입
- 미니배치 경사하강법을 실행하려면 매번 x,y,값을 바꿔줘여 한다.
- placeholder노드를 사용하여 실제로는 아무런 계산도하지 않는 특수한 노드이다. 
- 전형적으로 훈련을 하는 동안 텐서플로우에 훈련데이터를 주입하기 위해 사용한다. 

In [0]:
A=tf.placeholder(tf.float32, shape=(None,3)) #훈련데이터를 주입하기 위한다. 
B=A+5

#실행하는 session
with tf.Session() as sess:
  B_val_1=B.eval(feed_dict={A:[[1,2,3]]})
  B_val_2=B.eval(feed_dict={A:[[4,5,6],[7,8,9]]})

In [51]:
print(B_val_1)
print(B_val_2)

[[6. 7. 8.]]
[[ 9. 10. 11.]
 [12. 13. 14.]]


미니배치 경사하강법(전체 데이터를 가지고 경사하강법을 설정하는 것보단 효율적이면서 SGD의 단점을 극복하는 알고리즘이다.)을 구현하기 위해 구성단계에서 X,y의 정의를 placeholder 노드로 변경한다. 

In [0]:
#기본값을 설정하는 경우 
n_epochs = 1000
learning_rate = 0.01

In [0]:
X=tf.placeholder(tf.float32, shape=(None, n+1), name="X")
y=tf.placeholder(tf.float32, shape=(None,1), name="y")

In [0]:
#배치크기와 전체 배치 횟수를 지정한다. 
batch_size=100
n_batches=int(np.ceil(m/batch_size))

In [0]:
#노드를 평가할 때는 미니배치를 하나씩 추출하여 feed_dict매채변수로 전달
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

#연산을 실행하는 경우 
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

### 9.8.모델복원
- 모델파라미터를 디스크에 저장하는 방법이다.
- 체크포인트를 설정하여 다음 모델훈련의 파라미터를 결정하는데 참고가 되도록 한다. 
- 실행단계에서 모델을 저장하고 싶을 때 save()매서드에 체크포인트 파일의 경로를 저장한다 

### 9.9. 텐서보드로 그래프와 학습곡선 시각화 하기 
- 이제까지 미니배치 경사하강법을 사용하는 선형회귀 모델을 훈련시킬 계산 그래프를 만들었다. 그리고 일정한 간격으로 체크포인트를 저장한다. 
- 먼저 그래프 정의와 훈련 통계를 텐서보드가 읽을 수 있는 로그 디렉토르에 쓰도록 프로그램을 수정한다. 

In [0]:
from datetime import datetime

now=datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir="tf_logs"
logdir="{}/run-{}".format(root_logdir, now)

In [0]:
#미니배치 경사하강법을 구성한다
#기본적으로 노드를 형성하고 연결하는 과정을 거친다. 
n_epochs=1000
learning_rate=0.01

X=tf.placeholder(tf.float32, shape=(None, n+1), name="X")
y=tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta= tf.Variable(tf.random_uniform([n+1,1], -1.0, 1.0, seed=42), name="theta")
y_pred=tf.matmul(X, theta, name="predictions")
error=y_pred-y
mse=tf.reduce_mean(tf.square(error), name="mse")
optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op=optimizer.minimize(mse)

init=tf.global_variables_initializer()

In [0]:
#mse를 평가하고 텐서보드로 인식하는 summary
mse_summary=tf.summary.scalar('MSE', mse)

#객체를 만들고 로그디렉토리에 저장
file_writer=tf.summary.FileWriter(logdir, tf.get_default_graph())

In [0]:

n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [0]:
#그래프를 정의하여 작성하는 방법
with tf.Session() as sess:                                                        # not shown in the book
    sess.run(init)                                                                # not shown

    for epoch in range(n_epochs):                                                 # not shown
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()  

In [0]:
file_writer.close()

In [67]:
best_theta

array([[ 2.070016  ],
       [ 0.8204561 ],
       [ 0.11731729],
       [-0.22739056],
       [ 0.31134024],
       [ 0.00353192],
       [-0.01126995],
       [-0.9164395 ],
       [-0.8795009 ]], dtype=float32)

In [68]:
#tensorboard이외에 jupyter에서 시각화를 확인할 수 있는 방법
!pip install tfgraphviz

  Created wheel for tfgraphviz: filename=tfgraphviz-0.0.7-cp36-none-any.whl size=4432 sha256=a9e77ed7e81121f940795730400c22ba297cbe1ecd040042e190c414d91eeb2d
  Stored in directory: /root/.cache/pip/wheels/09/32/14/5a088574d718bfeb86629b01549771914c53a06b4363576b67
Successfully built tfgraphviz


In [70]:
import tensorflow as tf
import tfgraphviz as tfg

a = tf.constant(1, name="a")
b = tf.constant(2, name="b")
c = tf.add(a, b, name="add")
g = tfg.board(tf.get_default_graph())
g.view()

'G.gv.pdf'

### 9.10. 이름범위
신경망처럼 복잡한 모델을 다룰때는 계산그래프가 수천개의 노드에 의해 어질리거지기가 쉽다. 이를 피하기 위해 이름 범위를 만들어 관련 있는 노드들을 그룹으로 묶는다. 

In [0]:
#이전 코드를 사용하여 loss이름범위안에 있는 error와 mse를 정의한다. 
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")

In [0]:
#이름 loss라는 범위 내에 error와 mse노드를 추가하는 경우 
with tf.name_scope("loss") as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")

In [73]:
print(error.op.name)
print(mse.op.name)

loss/sub
loss/mse


### 9.11.모듈화
- ReLU함수는 sigmoid와 같은 일종의 활성함수이다. 
- binary의 경우에 내부 hidden layer을 활성화하는 함수로 sigmoid를 사용했다. 
- ReLU함수는 0보다 작은 값이 나오면 0을 반환하고 0이상의 값이 나오면 그 값을 출력하는 형태이다. 
![image](https://user-images.githubusercontent.com/49298791/74582068-233f7100-4ffa-11ea-8f44-6c641b7da0f6.png)

In [0]:
#2개의 ReLU함수를 정하는 그래프를 작성한다고 본다. 
n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")

w1 = tf.Variable(tf.random_normal((n_features, 1)), name="weights1")
w2 = tf.Variable(tf.random_normal((n_features, 1)), name="weights2")
b1 = tf.Variable(0.0, name="bias1")
b2 = tf.Variable(0.0, name="bias2")

z1 = tf.add(tf.matmul(X, w1), b1, name="z1")
z2 = tf.add(tf.matmul(X, w2), b2, name="z2")

relu1 = tf.maximum(z1, 0., name="relu1")
relu2 = tf.maximum(z2, 0., name="relu2")  # Oops, cut&paste error! Did you spot it?

output = tf.add(relu1, relu2, name="output")

In [0]:
def relu(X):
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_normal(w_shape), name="weights")
    b = tf.Variable(0.0, name="bias")
    z = tf.add(tf.matmul(X, w), b, name="z")
    return tf.maximum(z, 0., name="relu")

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

In [0]:
file_writer = tf.summary.FileWriter("logs/relu1", tf.get_default_graph())

In [0]:
#혹은 이름범위를 사용하여 정의도 가능하다. 
def relu(X):
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]), 1)                          # not shown in the book
        w = tf.Variable(tf.random_normal(w_shape), name="weights")    # not shown
        b = tf.Variable(0.0, name="bias")                             # not shown
        z = tf.add(tf.matmul(X, w), b, name="z")                      # not shown
        return tf.maximum(z, 0., name="max")                          # not shown

In [0]:

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

file_writer = tf.summary.FileWriter("logs/relu2", tf.get_default_graph())
file_writer.close()

### 9.12. 변수 공유
- 그래프의 여러 구성요소 간에 변수를 공유하고 싶다면 간단한 해결 방법은 변수를 먼저 만들고 함수에 매개변수로 전달하는 방법을 사용한다. 

In [0]:
def relu(X, threshold):
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]), 1)                        # not shown in the book
        w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
        b = tf.Variable(0.0, name="bias")                           # not shown
        z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
        return tf.maximum(z, threshold, name="max")

threshold = tf.Variable(0.0, name="threshold")
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X, threshold) for i in range(5)]
output = tf.add_n(relus, name="output")

In [0]:
def relu(X):
    with tf.name_scope("relu"):
        if not hasattr(relu, "threshold"):
            relu.threshold = tf.Variable(0.0, name="threshold")
        w_shape = int(X.get_shape()[1]), 1                          # not shown in the book
        w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
        b = tf.Variable(0.0, name="bias")                           # not shown
        z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
        return tf.maximum(z, relu.threshold, name="max")

In [0]:
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

In [0]:
def relu(X):
    with tf.variable_scope("relu", reuse=True):
        threshold = tf.get_variable("threshold")
        w_shape = int(X.get_shape()[1]), 1                          # not shown
        w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
        b = tf.Variable(0.0, name="bias")                           # not shown
        z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
        return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
with tf.variable_scope("relu"):
    threshold = tf.get_variable("threshold", shape=(),
                                initializer=tf.constant_initializer(0.0))
relus = [relu(X) for relu_index in range(5)]
output = tf.add_n(relus, name="output")